In [24]:
from matplotlib import pyplot as plt
from pprint import pprint

import json
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict, namedtuple
from p4utils.utils.helper import load_topo
import numpy as np

if not os.path.abspath('.').endswith('netscope'):
    root_path = os.path.dirname(os.path.dirname(os.path.abspath('.')))
    os.chdir(root_path)
    from analysis.load import Loader

In [25]:
# loader = Load.Loader()
# DF = loader.load2df()
# # topo = loader.get_topo()
# # df = df.drop(['meta_ingress_port', 'meta_egress_port', 'path_id3'], axis=1)
# DF

In [26]:
# log_dir = '/home/user/dds/netscope/experiment/data/slot2/ecmp_imbalance/20220930_0924GMT'
log_dir = "/mnt/netscope/data/slot2/fattree4/burst/20221005_1859GMT"
# log_dir = 'log'
loader = Loader(log_dir)
topo = loader.get_topo()
DF = loader.load_hosts(debug=True).sort_values('timestamp', ignore_index=True)
DF['flow'] = DF['src'] + '-' + DF['dst']

['h5-eth0-31.json', 'h2-eth0-31.json', 'h4-eth0-31.json', 'h7-eth0-31.json', 'h6-eth0-31.json', 'h3-eth0-31.json']


In [27]:
len(DF[DF.dst=='h5'])

507

In [28]:
DF.flow.unique()

array(['h1-h2', 'h2-h7', 'h3-h5', 'h5-h6', 'h6-h4', 'h7-h2', 'h1-h4',
       'h8-h3', 'h4-h5'], dtype=object)

In [29]:
EPOCH_SHIFT = 16
IN_DELAY = 20
PN_DELAY = 110
EN_DELAY = 60

EPOCH_LENGTH = 2**16  # microseconds
LBW = 100  # Mbps
Q_RATE = 9000  # packets/second (default: 3072) 8400 6300
Q_DEPTH_FRAC = 8
CTH_DEPTH_FRAC = 8  # 4 bw, 8 burst

# 【配置】队列总长度
Q_DEPTH = int(Q_RATE/Q_DEPTH_FRAC)  # packets (default: 128) Q_RATE/20 
# packets (1/4 of queue capacity, default: 32) Q_DEPTH/4
CTH_DEPTH = int(Q_DEPTH/CTH_DEPTH_FRAC)
# microseconds 10000 (default: 10000) SLA DEPENDENT
CTH_TIMEDELTA = int((1000000/Q_DEPTH_FRAC)/CTH_DEPTH_FRAC)

STH_BITRATE = int((LBW*1e6*0.5/8)/(1e6/EPOCH_LENGTH))

# df['epoch'] = df.timestamp.apply(lambda x: x >> EPOCH_SHIFT)

In [49]:
ddi = lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))) # dict dict int
dd0 = lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: np.array([0]*48)))) # dict dict zero
# sw - flow - epoch
i_pkts = ddi()
i_bytes = ddi()

e_contention_points = dd0()
e_suspicion_points = dd0()
e_high_delays = ddi()
e_ingress_packets = ddi()
e_egress_packets = ddi()
e_bytes = ddi()

df = DF.copy(deep=True)
timestamp_base = min(df.timestamp)

cal_epoch = lambda t: (t - timestamp_base) >> EPOCH_SHIFT

reports = []
e_epochs = set()
for i, row in df.iterrows():
    ########################
    # # Ingress
    ########################
    i_sw = row.debug[0]['sw_id']
    i_epoch = cal_epoch(row.debug[0]['timestamp'])
    i_pkts[i_sw][row.flow][i_epoch] += 1
    i_bytes[i_sw][row.flow][i_epoch] += row.debug[0]['packet_length']

    # ingress_packets = i_pkts[i_sw][row.flow][i_epoch]
    # ingress_bytes = i_bytes[i_sw][row.flow][i_epoch]


    ########################
    # # Egress
    ########################
    # INGRESS NODE PROCESSING (PART 2)
    e2e_delay = IN_DELAY
    contention_points = np.array([0]*48)
    suspicion_points = np.array([0]*48)

    # PROCESSING ON ALL NODES
    for path_length, hop in enumerate(row.debug):
        # INCREMENT FIELD: END-TO-END DELAY
        e2e_delay += PN_DELAY + hop['deq_timedelta']
        # CONTENTION?
        if hop['deq_timedelta'] >= CTH_TIMEDELTA or hop['enq_qdepth'] >= CTH_DEPTH:
            # MARK FIELD: CONTENTION POINTS
            contention_points[path_length] = 1
        # SUSPICION?
        if i_bytes[i_sw][row.flow][i_epoch] >= STH_BITRATE:
            suspicion_points[path_length] = 1
    df.loc[i, 'telemetry'] = [dict(
        contention_points=contention_points,
        suspicion_points=suspicion_points,
        e2e_delay=e2e_delay,
        i_epoch=i_epoch,
        i_packets=i_pkts[i_sw][row.flow][i_epoch],
        i_bytes=i_bytes[i_sw][row.flow][i_epoch],
        i_sw=i_sw,
    )]
    df.loc[i, 'e_timestamp'] = row.debug[-1]['timestamp']


for i, row in df.sort_values('e_timestamp').iterrows():
    # EGRESS NODE PROCESSING
    i_sw = row.telemetry['i_sw']
    i_epoch = row.telemetry['i_epoch']

    e_epoch = cal_epoch(row.debug[-1]['timestamp'])
    e_sw = row.debug[-1]['sw_id']
    e_epochs.add(e_epoch)
    e_contention_points[e_sw][row.flow][e_epoch] += row.telemetry['contention_points']
    e_suspicion_points[e_sw][row.flow][e_epoch] += row.telemetry['suspicion_points']
    e_bytes[e_sw][row.flow][e_epoch] += row.debug[-1]['packet_length']

    if row.telemetry['e2e_delay'] >= 20000:
        e_high_e2e_delay = 1
    else:
        e_high_e2e_delay = 0
    if i_epoch != e_epoch:
        e_high_delays[e_sw][row.flow][e_epoch] = e_high_e2e_delay
    elif e_high_e2e_delay == 1:
        e_high_delays[e_sw][row.flow][e_epoch] += 1
        
    # if row.telemetry['e2e_delay'] >= 20000 and i_epoch != e_epoch:
    #     print("row.telemetry['e2e_delay'] >= 20000")
    #     print(e_high_delays[e_sw][row.flow][e_epoch], i_epoch, e_epoch)

    e_drops = e_ingress_packets[e_sw][row.flow][e_epoch] - e_egress_packets[e_sw][row.flow][e_epoch]
    e_ingress_packets[e_sw][row.flow][e_epoch] = i_pkts[i_sw][row.flow][i_epoch]
    e_egress_packets[e_sw][row.flow][e_epoch] += 1

    
    # Clone
    if e_epoch != i_epoch and len(e_epochs) > 1:
        last_e_epoch = sorted(e_epochs)[-2] # 上一个epoch
        last_e_epoch = e_epoch
        path_len = row.debug
        r_tmp = dict(
            i_epoch=i_epoch,
            e_epoch=e_epoch,
            # last_e_epoch=last_e_epoch,
            flow=row.flow,
            path_id=row.path_id,
            contention_points=''.join(map(str, e_contention_points[e_sw][row.flow][last_e_epoch]))[:len(path_len)],
            suspicion_points=''.join(map(str, e_suspicion_points[e_sw][row.flow][last_e_epoch]))[:len(path_len)],
            high_delays=e_high_delays[e_sw][row.flow][last_e_epoch],
            delay=row.telemetry['e2e_delay'],
            drops=e_drops,
            i_bytes=row.telemetry['i_bytes'],
            e_bytes=e_bytes[e_sw][row.flow][e_epoch],
        )
        r_tmp.update(dict(
            whole_path=row.whole_path
        ))
        reports.append(r_tmp)

In [50]:
report = pd.DataFrame(reports)
for points in ['contention_points', 'suspicion_points']:
    report[points] = report[points].str.replace('0', '-')
report#.head()

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,whole_path
0,5,6,h2-h7,4,111,---,1,101651,0,265,321,"h2,s3,s2,s6,h7"
1,9,10,h2-h7,0,11-,---,1,67507,0,265,321,"h2,s3,s1,s6,h7"
2,14,15,h3-h5,4,111,---,1,101001,0,283,339,"h3,s4,s2,s5,h5"
3,17,19,h3-h5,0,11-,---,1,67316,0,283,339,"h3,s4,s1,s5,h5"
4,20,21,h2-h7,4,---,---,0,644,0,265,321,"h2,s3,s2,s6,h7"
...,...,...,...,...,...,...,...,...,...,...,...,...
623,1883,1884,h8-h3,1,---,---,0,584,0,338,394,"h8,s6,s1,s4,h3"
624,1887,1888,h8-h3,5,---,---,0,566,0,338,394,"h8,s6,s2,s4,h3"
625,1890,1891,h2-h7,0,---,---,0,564,0,265,321,"h2,s3,s1,s6,h7"
626,1893,1894,h4-h5,4,---,---,0,611,0,280,336,"h4,s4,s2,s5,h5"


In [51]:
report[report.flow=="h4-h5"]

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,whole_path
164,495,496,h4-h5,0,---,---,0,592,0,280,336,"h4,s4,s1,s5,h5"
171,518,519,h4-h5,4,---,---,0,594,0,280,336,"h4,s4,s2,s5,h5"
175,533,534,h4-h5,4,---,---,0,584,0,280,336,"h4,s4,s2,s5,h5"
179,541,542,h4-h5,4,---,---,0,599,0,280,336,"h4,s4,s2,s5,h5"
208,648,649,h4-h5,4,---,---,0,714,0,280,336,"h4,s4,s2,s5,h5"
209,652,653,h4-h5,0,---,---,0,576,0,280,336,"h4,s4,s1,s5,h5"
210,656,657,h4-h5,4,---,---,0,594,0,280,336,"h4,s4,s2,s5,h5"
216,683,684,h4-h5,0,---,---,0,589,0,280,336,"h4,s4,s1,s5,h5"
314,994,995,h4-h5,4,--1,---,1,21047,0,347,403,"h4,s4,s2,s5,h5"
316,1001,1002,h4-h5,0,--1,---,0,17467,0,347,403,"h4,s4,s1,s5,h5"


In [34]:
culprits = report[(report.drops>0) 
                | (report.high_delays > 0) 
                | (report.contention_points.str.strip('-') != "") 
                | (report.suspicion_points.str.strip('-') != "")]
culprits

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,whole_path


In [35]:
culprit_flows = sorted(culprits.value_counts(['whole_path']).items(), key=lambda x: x[1], reverse=True)
culprit_flows

[]

In [36]:
rca_list = []
for (whole_path, ), count in culprit_flows:
    path_l = whole_path.split(',')
    src, dst = path_l[0], path_l[-1]
    paths = topo.get_shortest_paths_between_nodes(src, dst)
    
    flow_cul_df = culprits[culprits.whole_path == whole_path]

    e_epochs = flow_cul_df.e_epoch.tolist()
    path_bytes = []
    for path in paths:
        path_df = report[report.whole_path == ",".join(path)]
        min_t, max_t = min(e_epochs), max(e_epochs)
        total_bytes = path_df[(path_df.e_epoch >= min_t) & (path_df.e_epoch <= max_t)].e_bytes.sum()
        path_bytes.append(total_bytes)
    path_bytes = np.array(path_bytes)/sum(path_bytes)
    rca_list.append(dict(
        culprit_path=whole_path,
        count=count,
        path_bytes=list(map(lambda x: f"{x:.2f}", path_bytes)),
        min_t=min_t,
        max_t=max_t,
        timespan=max_t - min_t,
    ))
pd.DataFrame(rca_list)

""


In [37]:

total_bytes = path_df[(path_df.e_epoch >= min(e_epochs)) & (path_df.e_epoch <= max(e_epochs))].e_bytes.sum()

In [38]:
for flow, report_flow in report.groupby('flow'):
    break
report_flow

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,whole_path
14,79,80,h1-h4,5,---,---,0,601,0,171,227,"h1,s3,s2,s4,h4"
15,83,84,h1-h4,1,---,---,0,591,0,171,227,"h1,s3,s1,s4,h4"
19,91,92,h1-h4,1,---,---,0,30081,0,171,227,"h1,s3,s1,s4,h4"
30,125,126,h1-h4,5,---,---,0,31129,0,171,227,"h1,s3,s2,s4,h4"
31,129,130,h1-h4,1,---,---,0,20347,0,171,227,"h1,s3,s1,s4,h4"
...,...,...,...,...,...,...,...,...,...,...,...,...
613,1840,1841,h1-h4,1,---,---,0,592,0,171,227,"h1,s3,s1,s4,h4"
615,1844,1845,h1-h4,5,---,---,0,590,0,171,227,"h1,s3,s2,s4,h4"
616,1852,1853,h1-h4,5,---,---,0,598,0,171,227,"h1,s3,s2,s4,h4"
617,1856,1857,h1-h4,1,---,---,0,568,0,171,227,"h1,s3,s1,s4,h4"


In [39]:
report[report.high_delays>1]

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,whole_path
